In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import PySimpleGUI as sg
import pyWinhook as pyHook
import pythoncom

In [2]:
data = pd.read_csv('keystroke.csv')

X = data.iloc[:, 3:]
y = data['subject']

# divide data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Create a Random Forest model
randomForestModel = RandomForestClassifier()

# Train the model
randomForestModel.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = randomForestModel.score(X_test, y_test)
print('Test accuracy:', accuracy)

Test accuracy: 0.9304262616364527


In [4]:
class KeyLogger:
    def __init__(self):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        self.userName = getUserName()
        self.idOfWord = 1
        #self.message = ""

    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False:
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True:
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
    

    #Currently not used
    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

            
    def keyUpEvent(self, event):
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        print(chr(event.Ascii),end='')
        self.storeEvent("Up", event)
        return True


    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time)

        # Makes sure the enter key up and down have the same idOfWord
        if event.Ascii == 13 and activity == "Up":
            self.eventList.append ((self.userName, self.idOfWord - 1, event.Ascii, event.Key, activity, int(keystrokeTime)))
        else:
            self.eventList.append ((self.userName, self.idOfWord, event.Ascii, event.Key, activity, int(keystrokeTime)))

       
        
        
        # Chosen to use Escape key (ESC) due to input using a similar method
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        # Activity condition had to be added because the userRecordData function was being called twice because of the keyUpEvent and keyDownEvent
        # if event.Ascii == 13 and activity == "Down": 
        #     self.enterPressed = True
        #     print("Enter Pressed")
        #     userRecordData(self.eventList)

def userRecordData(eventList):
    # Delete the first element of the list because the first element is always a keyUpEvent for the Enter key
    eventList.pop(0)
    
    df = pd.DataFrame(eventList, columns = ['user', 'idOfWord', 'ascii', 'key', 'keyEvent', 'Time'])
    df = df[df.key != 'R']
    print(df)

    
    df2 = pd.DataFrame(columns=['subject','idOfWord','key','H','UD','DD'])

    for i in range(0, len(df), 2):
        finalData = {}
        print(i)

        finalData['subject'] = df.iloc[i].user
        finalData['key'] = df.iloc[i].key
        finalData['H'] = (int(df.iloc[i+1].Time) - int(df.iloc[i].Time))/1000
        finalData['idOfWord'] = df.iloc[i].idOfWord
        
        if(df.iloc[i].key != "Return"):
            finalData['UD'] = (int(df.iloc[i+2].Time) - int(df.iloc[i+1].Time))/1000
            finalData['DD'] = (int(df.iloc[i+2].Time) - int(df.iloc[i].Time))/1000
        
        df2 = pd.concat([df2, pd.DataFrame(finalData, index=[0])], ignore_index=True)
        
    print(df2)


#Not needed currently
def getKeyStroke():
    
    keyLogger = KeyLogger()
    hookManager = pyHook.HookManager()
    hookManager.KeyDown = keyLogger.keyDownEvent
    hookManager.KeyUp = keyLogger.keyUpEvent
    hookManager.HookKeyboard()

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()


def getUserName():
        userName = input("Enter your name: ")
        return userName

In [5]:
sg.theme('DarkAmber')   # Add a touch of color

#TODO: 1) Make it so when the username is entered the Up event of the enter key is not recorded

# Create the keylogger and hook it to the keyboard
keyLogger = KeyLogger()
hookManager = pyHook.HookManager()
hookManager.KeyDown = keyLogger.keyDownEvent
hookManager.KeyUp = keyLogger.keyUpEvent
hookManager.HookKeyboard()

# All the stuff inside your window.
layout = [  [sg.Text('Collecting KeyStroke Data for User: ' + keyLogger.userName)],
            [sg.Text('Enter password (.tie5Roanl)'), sg.InputText( key='Input1')],
            [sg.Button('Login'), sg.Button('Cancel')] ]

# Create the Window
window = sg.Window('Window Title', layout, finalize=True)

# Bind the Enter key to the input field -> Used to clear the input field when enter is pressed
# For this to work, when creating the window, finalize must be set to True
window['Input1'].bind('<Return>', '_Enter')

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    elif event == 'Login': # This implementation prevents the same data from being recorded twice
        userRecordData(keyLogger.eventList)
        break
    
    #keyLogger.mainLoop()
    pythoncom.PumpWaitingMessages() # This is needed for the keylogger to work, and works without the mainLoop function

    #print('You entered ', values[0])
    print(values)

# Unhooks the keyboard, no more data recorded, returns to menu
hookManager.UnhookKeyboard()
window.close()

.tie5R oanl     user  idOfWord  ascii         key keyEvent       Time
0   klara         1     46  Oem_Period     Down  807614203
1   klara         1     46  Oem_Period       Up  807614328
2   klara         1    116           T     Down  807614359
3   klara         1    116           T       Up  807614500
4   klara         1    105           I     Down  807614562
5   klara         1    105           I       Up  807614656
6   klara         1    101           E     Down  807614843
7   klara         1    101           E       Up  807614984
8   klara         1     53           5     Down  807615406
9   klara         1      0      Lshift     Down  807615515
10  klara         1     53           5       Up  807615531
11  klara         1     82           R     Down  807615703
12  klara         1     82           R       Up  807615843
13  klara         1      0      Lshift       Up  807615859
14  klara         1    111           O     Down  807615875
15  klara         1    111           O       

IndexError: single positional indexer is out-of-bounds

: 

In [ ]:
#accuracy of 0.72
# from sklearn.linear_model import LogisticRegression

# # Create a Logistic Regression model
# model = LogisticRegression(max_iter=2000)

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)


In [ ]:
#accuracy of 0.84
# from sklearn.preprocessing import StandardScaler

# # Scale the input features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Create a Logistic Regression model and train it on the scaled data
# model = LogisticRegression(solver='sag', max_iter=5000)
# model.fit(X_train_scaled, y_train)

# # Evaluate the model on the scaled test set
# accuracy = model.score(X_test_scaled, y_test)
# print('Test accuracy:', accuracy)


In [ ]:
#accuracy of 0.72
# from sklearn.tree import DecisionTreeClassifier

# # Create a Decision Tree model
# model = DecisionTreeClassifier()

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)


In [ ]:
#accuracy of 0.79
# from sklearn.svm import SVC

# # Create an SVM model
# model = SVC()

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)


In [ ]:
#accuracy of 0.74
# from sklearn.neighbors import KNeighborsClassifier

# # Create a KNN model
# model = KNeighborsClassifier()

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)